In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from apyori import apriori

In [2]:
# Loading the data 

# Loading Aisles data into Aisles
aisles = pd.read_csv('aisles.csv')

# Loading Departments data into Departments
departments = pd.read_csv('departments.csv')

# Loading Training order data into orderProductsTrain
orderProductsTrain = pd.read_csv('order_products__train.csv')

# Loading the Orders into Orders
orders = pd.read_csv('orders.csv')

# Loading Products data into Products
products = pd.read_csv('products.csv')

# Loading Prior order data into orderProductsPrior
orderProductsPrior = pd.read_csv('order_products__prior.csv')

In [3]:
# checking if dfs have nulls one by one
aisles.isnull().sum()
departments.isnull().sum()
products.isnull().sum()
orderProductsPrior.isnull().sum()
orderProductsTrain
orders.isnull().sum() # orders dataset has some nulls. We will drop them.

order_id                       0
user_id                        0
eval_set                       0
order_number                   0
order_dow                      0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64

In [4]:
orders.dropna(inplace=True)

In [5]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
5,3367565,1,prior,6,2,7,19.0


In [6]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [7]:
orderProductsTrain.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [8]:
orderProductsPrior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [9]:
departments.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [10]:
aisles.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [11]:
# Merging tables together.
products_departments = products.merge(departments, left_on='department_id', right_on='department_id', how='left')
products_departments_aisles = products_departments.merge(aisles, left_on='aisle_id', right_on='aisle_id', how='left')

products_departments_aisles.groupby('department')['product_id'].count().reset_index().sort_values(by='product_id', ascending=False).head(10)


# Merging products_departments_aisles and order_products_total.
order_products_total = pd.concat([orderProductsPrior, orderProductsTrain]) 
df = order_products_total.merge(products_departments_aisles, left_on='product_id', right_on='product_id', how='left')
df.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,department,aisle
0,2,33120,1,1,Organic Egg Whites,86,16,dairy eggs,eggs
1,2,28985,2,1,Michigan Organic Kale,83,4,produce,fresh vegetables
2,2,9327,3,0,Garlic Powder,104,13,pantry,spices seasonings
3,2,45918,4,1,Coconut Butter,19,13,pantry,oils vinegars
4,2,30035,5,0,Natural Sweetener,17,13,pantry,baking ingredients


## Association Rule Mining

In [35]:
# Train set
train_df = df.copy()
data = train_df[['order_id', 'product_id']]
data.head(15)

,order_id,product_id
0,2,33120
1,2,28985
2,2,9327
3,2,45918
4,2,30035
5,2,17794
6,2,40141
7,2,1819
8,2,43668
9,3,33754


In [14]:
data.shape

(33819106, 2)

In [34]:
# grouping products based on order_id
products_per_order_df = data.groupby('order_id').apply(lambda x: ",".join(x.product_id.astype(str))).to_frame()
products_per_order_df.head(15)

,0
order_id,
1,"49302,11109,10246,49683,43633,13176,47209,22035"
2,"33120,28985,9327,45918,30035,17794,40141,1819,..."
3,"33754,24838,17704,21903,17668,46667,17461,32665"
4,"46842,26434,39758,27761,10054,21351,22598,3486..."
5,"13176,15005,47329,27966,23909,48370,13245,9633..."
6,"40462,15873,41897"
7,"34050,46802"
8,23423
9,"21405,47890,11182,2014,29193,34203,14992,31506..."


In [15]:
products_per_order_df.shape

(3346083, 1)

In [16]:
# reformatting the records suitable for input into the apriori model
records = [row[0].split(",") for idx, row in products_per_order_df.iterrows()]
len(records)

3346083

In [17]:
records

[['49302', '11109', '10246', '49683', '43633', '13176', '47209', '22035'],
 ['33120',
  '28985',
  '9327',
  '45918',
  '30035',
  '17794',
  '40141',
  '1819',
  '43668'],
 ['33754', '24838', '17704', '21903', '17668', '46667', '17461', '32665'],
 ['46842',
  '26434',
  '39758',
  '27761',
  '10054',
  '21351',
  '22598',
  '34862',
  '40285',
  '17616',
  '25146',
  '32645',
  '41276'],
 ['13176',
  '15005',
  '47329',
  '27966',
  '23909',
  '48370',
  '13245',
  '9633',
  '27360',
  '6348',
  '40878',
  '6184',
  '48002',
  '20914',
  '37011',
  '12962',
  '45698',
  '24773',
  '18569',
  '41176',
  '48366',
  '47209',
  '46522',
  '38693',
  '48825',
  '8479'],
 ['40462', '15873', '41897'],
 ['34050', '46802'],
 ['23423'],
 ['21405',
  '47890',
  '11182',
  '2014',
  '29193',
  '34203',
  '14992',
  '31506',
  '23288',
  '44533',
  '18362',
  '27366',
  '432',
  '3990',
  '14183'],
 ['24852',
  '4796',
  '31717',
  '47766',
  '4605',
  '1529',
  '21137',
  '22122',
  '34134',
  '2

In [18]:
# build the model - min_support value can be decreased for more association rules
association_rules = apriori(records, min_support=0.001, min_confidence=0.10, min_lift= 2, min_lenght=2)
type(association_rules)

generator

In [19]:
# converting the generator into a list
association_rules_lst = list(association_rules)

print('Length of association rules extracted:')
print(len(association_rules_lst))
print('\nGlimpse of few association rules:')
print(association_rules_lst[:5])

Length of association rules extracted:
884

Glimpse of few association rules:
[RelationRecord(items=frozenset({'10132', '21137'}), support=0.0011631510634972295, ordered_statistics=[OrderedStatistic(items_base=frozenset({'10132'}), items_add=frozenset({'21137'}), confidence=0.20772843723313408, lift=2.5222590870876633)]), RelationRecord(items=frozenset({'21903', '10246'}), support=0.0013965583041424855, ordered_statistics=[OrderedStatistic(items_base=frozenset({'10246'}), items_add=frozenset({'21903'}), confidence=0.18776117004178722, lift=2.4960348786751694)]), RelationRecord(items=frozenset({'22935', '10246'}), support=0.0010953105466899656, ordered_statistics=[OrderedStatistic(items_base=frozenset({'10246'}), items_add=frozenset({'22935'}), confidence=0.14725972356155576, lift=4.185864772792324)]), RelationRecord(items=frozenset({'47209', '10246'}), support=0.0010558614355949928, ordered_statistics=[OrderedStatistic(items_base=frozenset({'10246'}), items_add=frozenset({'47209'}), co

In [30]:
# Visualising the model output in a tabular form
def inspect(output):
    lhs         = [tuple(result[2][0][0])[0] if len(tuple(result[2][0][0])) > 0 else "" for result in output]
    rhs         = [tuple(result[2][0][1])[0] for result in output]
    support    = [result[1] for result in output]
    confidence = [result[2][0][2] for result in output]
    lift       = [result[2][0][3] for result in output]
    return list(zip(lhs, rhs, support, confidence, lift))

output_DataFrame = pd.DataFrame(inspect(association_rules_lst), columns = ['LHS', 'RHS', 'Support', 'Confidence', 'Lift'])

# replacing product_id(s) with their product_name(s)
LHS_product_name = []
for each_prod_id in output_DataFrame['LHS']:
    LHS_product_name.append(products[products.product_id == int(each_prod_id)]['product_name'].item())
RHS_product_name = []
for each_prod_id in output_DataFrame['RHS']:
    RHS_product_name.append(products[products.product_id == int(each_prod_id)]['product_name'].item())
output_DataFrame['LHS'] = LHS_product_name
output_DataFrame['RHS'] = RHS_product_name

print('\nGlimpse of the model output in tabular form:')
print(output_DataFrame.head())


Glimpse of the model output in tabular form:
                               LHS                   RHS   Support  \
0  Organic American Cheese Singles  Organic Strawberries  0.001163   
1            Organic Celery Hearts  Organic Baby Spinach  0.001397   
2            Organic Celery Hearts  Organic Yellow Onion  0.001095   
3            Organic Celery Hearts  Organic Hass Avocado  0.001056   
4          Organic Red Bell Pepper  Organic Strawberries  0.003727   

   Confidence      Lift  
0    0.207728  2.522259  
1    0.187761  2.496035  
2    0.147260  4.185865  
3    0.141956  2.150502  
4    0.208273  2.528877  


In [25]:
# saving the model output
output_DataFrame.to_csv("recommendations_884_pn.csv")

In [27]:
# reading the association rules from the saved model output
recommendation_engine = pd.read_csv("recommendations_884_pn.csv")
recommendation_engine.head(10)

,Unnamed: 0,LHS,RHS,Support,Confidence,Lift
0,0,Organic American Cheese Singles,Organic Strawberries,0.001163,0.207728,2.522259
1,1,Organic Celery Hearts,Organic Baby Spinach,0.001397,0.187761,2.496035
2,2,Organic Celery Hearts,Organic Yellow Onion,0.001095,0.147260,4.185865
3,3,Organic Celery Hearts,Organic Hass Avocado,0.001056,0.141956,2.150502
4,4,Organic Red Bell Pepper,Organic Strawberries,0.003727,0.208273,2.528877
5,5,Organic Red Bell Pepper,Organic Baby Spinach,0.003080,0.172117,2.288062
6,6,Organic Red Bell Pepper,Green Bell Pepper,0.002118,0.118341,6.826611
7,7,Organic Red Bell Pepper,Organic Yellow Onion,0.002367,0.132252,3.759277
8,8,Organic Red Bell Pepper,Organic Garlic,0.001859,0.103911,3.051676
9,9,Organic Red Bell Pepper,Limes,0.002027,0.113297,2.584899


In [31]:
# validate data
def recommendations_by_product_name(product_name : str):
    """ Function to return recommendations based on input product id """
    if product_name not in recommendation_engine.LHS.unique():
        return "Sorry no recommendations"
    else:
        return list(set(recommendation_engine[recommendation_engine.LHS == product_name]['RHS'].tolist()))

In [32]:
# Experiment 1 - get recommendation for a selected product
selected_product = 'Limes'
print('\nProduct recommendations on selecting product - ', selected_product)
recommendations_by_product_name(selected_product)


Product recommendations on selecting product -  Limes


['Cucumber Kirby',
 'Organic Avocado',
 'Organic Hass Avocado',
 'Large Lemon',
 'Organic Cilantro',
 'Organic Lemon']

In [33]:
# Experiment 2 - get recommendation for a selected product
selected_product = 'Organic Red Bell Pepper'
print('\nProduct recommendations on selecting product - ', selected_product)
recommendations_by_product_name(selected_product)


Product recommendations on selecting product -  Organic Red Bell Pepper


['Organic Baby Spinach',
 'Organic Cucumber',
 'Green Bell Pepper',
 'Organic Strawberries',
 'Organic Yellow Onion',
 'Organic Garlic',
 'Organic Avocado',
 'Organic Bell Pepper',
 'Organic Hass Avocado',
 'Organic Raspberries',
 'Limes',
 'Organic Zucchini']